In [1]:
# Dependencies
import requests
import json
import pandas as pd
from flask import jsonify, make_response
from json import dumps

#importing from api.py 
from api import apiEndpoint, apiKey, base_url, geoapify_key

In [2]:
#getting the state data to put the states into a list

response = requests.get(f'{apiEndpoint}?apiKey={apiKey}')



if response.status_code == 200:
    # Try to serialize the response object to JSON
    try:
        data = json.loads(response.content)
        print(data)
    except Exception as e:
        # If serialization fails, convert non-serializable data to a JSON-serializable format
        if isinstance(response.content, bytes):
            data = response.content.decode('utf-8')
            print(data)
        else:
            print(f"Failed to serialize response: {e}")
else:
    print(f"Request failed with status code: {response.status_code}")

[{'fips': '02', 'country': 'US', 'state': 'AK', 'county': None, 'hsa': None, 'hsaName': None, 'level': 'state', 'lat': None, 'locationId': 'iso1:us#iso2:us-ak', 'long': None, 'population': 731545, 'hsaPopulation': None, 'metrics': {'testPositivityRatio': 0.1, 'testPositivityRatioDetails': {'source': 'other'}, 'caseDensity': 7.0, 'weeklyNewCasesPer100k': 49.1, 'contactTracerCapacityRatio': None, 'infectionRate': 1.0, 'infectionRateCI90': 0.18, 'icuCapacityRatio': 0.74, 'bedsWithCovidPatientsRatio': 0.013, 'weeklyCovidAdmissionsPer100k': 4.4, 'vaccinationsInitiatedRatio': 0.732, 'vaccinationsCompletedRatio': 0.652, 'vaccinationsAdditionalDoseRatio': 0.324, 'vaccinationsFall2022BivalentBoosterRatio': 0.139}, 'riskLevels': {'overall': 1, 'testPositivityRatio': 1, 'caseDensity': 1, 'contactTracerCapacityRatio': 4, 'infectionRate': 1, 'icuCapacityRatio': 1}, 'cdcTransmissionLevel': 3, 'communityLevels': {'cdcCommunityLevel': None, 'canCommunityLevel': 0}, 'actuals': {'cases': 296561, 'deaths

In [3]:


# Check the response status code
if response.status_code == 200:
    # Parse the JSON response and extract the list of states
    data = response.json()
    states = [state['state'] for state in data]
    print(states)
else:
    print(f"Request failed with status code: {response.status_code}")

['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MP', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'PR', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']


In [4]:
# testing with one city first
target_city = "Northern Marina Islands"

params = {
    "text": target_city,
    "apiKey": geoapify_key
}

base_url = "https://api.geoapify.com/v1/geocode/search"

# Run request
response = requests.get(base_url, params=params).json()

# Print the json (pretty printed)
print(json.dumps(response, indent=4, sort_keys=True))

{
    "features": [
        {
            "bbox": [
                145.7585,
                15.214,
                145.7585,
                15.214
            ],
            "geometry": {
                "coordinates": [
                    145.7585,
                    15.214
                ],
                "type": "Point"
            },
            "properties": {
                "address_line1": "Micronesia - Northern Mariana Islands, Northern Mariana Islands",
                "address_line2": "United States of America",
                "city": "Micronesia - Northern Mariana Islands",
                "country_code": "mp",
                "datasource": {
                    "sourcename": "whosonfirst",
                    "url": "https://www.whosonfirst.org/docs/licenses/"
                },
                "dependency": "Northern Mariana Islands",
                "formatted": "Micronesia - Northern Mariana Islands, Northern Mariana Islands, United States of America",
        

In [5]:
# Extract lat/lon
lat = response["features"][0]["properties"]["lat"]
lon = response["features"][0]["properties"]["lon"]

# Print results
print(f"{target_city}: {lat}, {lon}")

Northern Marina Islands: 15.214, 145.7585


In [6]:
lat_list = []
lon_list = []
states = ['Alaska', 'Alabama', 'Arkansas', 'Arizona', 'California', 'Colorado', 'Connecticut', 'District of Columbia', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Iowa', 'Idaho', 'Illinois', 'Indiana', 'Kansas', 'Kentucky', 'Louisiana', 'Massachusetts', 'Maryland', 'Maine', 'Michigan', 'Minnesota', 'Missouri', 'Northern Mariana Islands', 'Mississippi', 'Montana', 'North Carolina', 'North Dakota', 'Nebraska', 'New Hampshire', 'New Jersey', 'New Mexico', 'Nevada', 'New York', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Puerto Rico', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Virginia', 'Vermont', 'Washington', 'Wisconsin', 'West Virginia', 'Wyoming']
for state in states:
    # Add the state to the parameters for the API request:
    target_state = state
    
    params = {
        "text":target_state,
        "apiKey": geoapify_key
    }
    response2 = requests.get(base_url, params=params).json()
    
    lat = response2['features'][0]['properties']['lat']
    lng = response2['features'][0]['properties']['lon']
    
    lat_list.append(lat)
    lon_list.append(lng)

In [7]:
latlon_df = pd.DataFrame({
    'latitude': lat_list,
    'longitude': lon_list
})

In [18]:
latlon_df

#finding states where geoapify was giving me wrong coordinates
states.index("New York")
states.index("Georgia")

10

In [12]:
#coordinates from geoapify were wrong for a few states D:

latlon_df.loc[latlon_df.index[-4], "latitude"] = 47.273015
latlon_df.loc[latlon_df.index[-4], "longitude"] = -120.88227

latlon_df.loc[latlon_df.index[-1], "latitude"] = 43.07597
latlon_df.loc[latlon_df.index[-1], "longitude"] = -107.290283

latlon_df.loc[35, "latitude"] = 42.7466
latlon_df.loc[35, "longitude"] = -75.77

latlon_df.loc[10, "latitude"] = 33.24788
latlon_df.loc[10, "longitude"] = -83.441162

latlon_df

,latitude,longitude
0,64.445961,-149.680909
1,33.258882,-86.829534
2,35.204888,-92.447911
3,34.395342,-111.763275
4,36.701463,-118.755997
5,38.725178,-105.607716
6,41.650020,-72.734216
7,38.893847,-76.988043
8,38.692045,-75.401331
9,28.542111,-81.379030


In [20]:
df_json = latlon_df.to_json("latlon.json", orient = 'values')
df_json

In [14]:
df_csv = latlon_df.to_csv("latlon.csv", index = False)